In [2]:
import pandas as pd
from pathlib import Path
import re
import subprocess
import numpy as np

import locations
from runner import tags

from ipywidgets import widgets, Layout, fixed
import plotly.express as px

In [34]:
non_static_df = pd.read_csv(locations.OUTDIR/"coverage_by_tag_ns_nt_i.csv").rename(columns={"non_static": "functions"})
static_df = pd.read_csv(locations.OUTDIR/"coverage_by_tag_s_nt_i.csv")
df = static_df
df["ratio"] = df["tested"]/df["functions"]
df["directory"].fillna('', inplace=True)
df

,directory,functions,tested,tag,ratio
0,,275215,994,v3.0,0.003612
1,Documentation,263,5,v3.0,0.019011
2,arch,42320,214,v3.0,0.005057
3,block,768,0,v3.0,0.000000
4,crypto,1111,36,v3.0,0.032403
...,...,...,...,...,...
118227,drivers/staging/media/atomisp/pci/isp/kernels/...,3,0,v5.11,0.000000
118228,drivers/staging/media/atomisp/pci/isp/kernels/...,5,0,v5.11,0.000000
118229,drivers/staging/media/atomisp/pci/isp/kernels/...,6,0,v5.11,0.000000
118230,drivers/staging/media/atomisp/pci/isp/kernels/...,7,0,v5.11,0.000000


In [17]:
def make_plot(df, directory):
    directory=directory.rstrip("/")
    ax = df[df["directory"] == directory].plot(
        x="tag", y=["functions", "tested", "ratio"], secondary_y="ratio", title=directory or 'whole kernel',
        figsize=(10, 10), fontsize=16
    )
    ax.set_xlabel("Version Number", fontdict={'fontsize':16})
    ax.set_ylabel("Number of Functions", fontdict={'fontsize':16})
    ax.right_ax.set_ylabel("Coverage Ratio", fontdict={'fontsize':16})
    bottom, top = ax.get_ylim()
    ax.set_ylim(0, top)
    bottom, top = ax.right_ax.get_ylim()
    ax.right_ax.set_ylim(0, top)

In [18]:
# top_level = ['','Documentation','arch','block','crypto','drivers','firmware','fs',
# 'init','ipc','kernel','lib','mm','net','samples','scripts','security','sound','tools','usr','virt']


# for directory in top_level:
#     make_plot(df, directory)

In [19]:
def interactive_plot(df):
    selector = widgets.Combobox(options=list(set(df["directory"])), description="directory",
                        layout=Layout(width="500px"))
    plot = widgets.interactive_output(make_plot, {'df': fixed(df), 'directory':selector})
    display(plot, selector)

In [20]:
# todo try log scale

In [21]:
interactive_plot(df)

Output()

Combobox(value='', description='directory', layout=Layout(width='500px'), options=('', 'arch/microblaze/mm', '…

In [22]:
nonzero_filter = df.groupby("directory").sum()["tested"] > 0
nonzero_test_dirs = list(nonzero_filter[nonzero_filter].index)
nonzero_tests = df[df["directory"].apply(lambda d: d in nonzero_test_dirs)]

In [23]:
interactive_plot(nonzero_tests)

Output()

Combobox(value='', description='directory', layout=Layout(width='500px'), options=('', 'arch/microblaze/mm', '…

In [31]:
def prep_treedf(df, version):
    treedf = df.copy()
    treedf = treedf[treedf.tag == version]
    treedf["directory"].replace("", "kernel root", inplace=True)

    treedf["parent"] = treedf["directory"].apply(get_parent)
    treedf["count excluding subdirectories"] = treedf["functions"]

    rows = treedf.copy().iterrows()
    for parent,value in treedf.groupby("parent")["functions"].sum().iteritems():
        treedf.loc[treedf["directory"] == parent,["count excluding subdirectories"]] -= value

    assert len(treedf[treedf["count excluding subdirectories"] < 0]) == 0
    return treedf

def treemap(df, version):
    treedf = prep_treedf(df, version)
    color = treedf["ratio"].replace(0, np.nan)
    fig = px.treemap(treedf, names="directory", parents="parent",
                     values="count excluding subdirectories", color=color,
                     color_continuous_scale=["red", "yellow", "green"],
                     height=600,
                     range_color=(0,0.1), # we can cap the max ratio colour
                     hover_data=["ratio", "functions"]
                    )
    return fig

def show_treemap(df, version):
    treemap(df, version).show()
    
def get_parent(p):
    if p == "kernel root":
        return ""
    if "/" not in p:
        return "kernel root"
    return "/".join(p.split("/")[:-1])


def treemap_with_selector(df):
    selector = widgets.Dropdown(options=[x for x,_,_ in tags()], description="Kernel Version")
    plot = widgets.interactive_output(show_treemap, {'df': fixed(df), 'version':selector})
    return (selector, plot)

In [32]:
display(*treemap_with_selector(df))

Dropdown(description='Kernel Version', options=('v3.0', 'v3.1', 'v3.2', 'v3.3', 'v3.4', 'v3.5', 'v3.6', 'v3.7'…

Output()

In [33]:
display(*treemap_with_selector(df))

Dropdown(description='Kernel Version', options=('v3.0', 'v3.1', 'v3.2', 'v3.3', 'v3.4', 'v3.5', 'v3.6', 'v3.7'…

Output()

# TODO: URL